## Neural Network Learning

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [6]:
X= np.loadtxt('Files/ex3data1.txt')
y= np.loadtxt('Files/ex3datay.txt')
print(X.shape)
print(y.shape)

(5000, 400)
(5000,)


In [8]:
theta1= np.loadtxt('Files/ex3weights.txt')
theta2= np.loadtxt('Files/ex3weights2.txt')
print(theta1.shape)
print(theta2.shape)

(25, 401)
(10, 26)


In [9]:
def display(index):
    ex= X[index]
    ex= ex.reshape((20,20)).transpose()
    plt.imshow(ex, cmap= 'gray')

In [10]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z));

In [42]:
def fwp(index, l, theta1, theta2):
    x= X[index]
    x= x[None].transpose()
    if(l==1): 
        return x
    
    a1= np.insert(x, 0, 1, axis=0)
    z2= np.dot(theta1, a1)
    a2= sigmoid(z2)
    a2= np.insert(a2, 0, 1, axis=0)
    if(l==2):
        return a2
    
    z3= np.dot(theta2, a2)
    a3= sigmoid(z3)
    
    return a3

In [12]:
fwp(1,3,theta1, theta2).shape

(10, 1)

In [13]:
def hyp(index, theta1, theta2):
    x= X[index]
    x= x[None].transpose()
    
    a1= np.insert(x, 0, 1, axis=0)
    z2= np.dot(theta1, a1)
    a2= sigmoid(z2)
    
    a2= np.insert(a2, 0, 1, axis=0)
    z3= np.dot(theta2, a2)
    a3= sigmoid(z3)
    
    return a3

In [14]:
def predict(index, info=False):
    x= X[index]
    x= x[None].transpose()
    
    a1= np.insert(x, 0, 1, axis=0)
    z2= np.dot(theta1, a1)
    a2= sigmoid(z2)
    
    a2= np.insert(a2, 0, 1, axis=0)
    z3= np.dot(theta2, a2)
    a3= sigmoid(z3)
    
    output= np.argmax(a3) + 1
    
    if(info):
        #print(a3,'\n')
        print("True value is ",y[index])
        print("Predicted value is ",output)
        print("Probablity is ", max(a3))
        display(index)
    return output==y[index]

In [15]:
input_layer_size= 400
hidden_layer_size= 25
num_labels= 10
yy= pd.Series(y)
ys= pd.get_dummies(yy)
ys.iloc[0].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [107]:
theta1= np.random.rand(hidden_layer_size, input_layer_size)
theta2= np.random.rand(num_labels, hidden_layer_size)
theta1= np.insert(theta1, 0, 1, axis=1)
theta2= np.insert(theta2, 0, 1, axis=1)
print(theta1.shape)
print(theta2.shape)

(25, 401)
(10, 26)


In [143]:
theta= np.array([theta1, theta2])
print(theta.shape)
theta[1].shape


(2,)


(10, 26)

### Cost Function

In [144]:
def costFunc(theta):
    m= 5000
    ans=0
    theta1= theta[0]
    theta2= theta[1]
    for i in range(m):
        h= hyp(i, theta1, theta2)
        y= ys.iloc[i].values[None]

        t1= np.dot(y,np.log(h))
        t2= np.dot((1-y), np.log(1- h))
        ans+= (t1+t2)
        
    ans*= (-1.0/m)
    return ans


### Gradient Computation

In [145]:
def gradient(theta):
    theta1= theta[0]
    theta2= theta[1]
    
    cdel1= np.zeros((25,401))
    cdel2= np.zeros((10,26))
    for i in range(10):
        x= X[i]
        x= x[None].transpose()
        yt= ys.iloc[i].values[None].transpose()
        #print('yt', yt.shape)

        a1= x
        #print('a1', a1.shape)

        a1= np.insert(x, 0, 1, axis=0)
        z2= np.dot(theta1, np.insert(x, 0, 1, axis=0))
        a2= sigmoid(z2)
        #print('a2', a2.shape)

        a2= np.insert(a2, 0, 1, axis=0)
        z3= np.dot(theta2, a2)
        a3= sigmoid(z3)
        #print('a3', a3.shape)

        del3= a3 - yt
        #print('del3',del3.shape)

        del2= np.dot(theta2.transpose(), del3) * a2 * (1-a2)
        #print('del2',del2.shape)

        cdel2+= np.dot(del3, a2.transpose())
        #print('cdel2',cdel2.shape)
        cdel1+= np.dot(del2[1:,:], a1.transpose())
        #print('cdel1',cdel1.shape)

    print('*'*50)
    print('theta1', theta1.shape)
    print('theta2', theta2.shape)
    d2= (cdel2 + theta2) / m
    print('d2', d2.shape)
    d1= (cdel1 + theta1) / m
    print('d1', d1.shape)
    
    d= np.array([d1, d2])
    return d
#gradient()

In [149]:
theta1= np.random.rand(hidden_layer_size, input_layer_size)
theta2= np.random.rand(num_labels, hidden_layer_size)
theta1= np.insert(theta1, 0, 1, axis=1)
theta2= np.insert(theta2, 0, 1, axis=1)
initTheta= np.array([theta1, theta2])
print(theta1.shape)
print(theta2.shape)
print(initTheta.shape)

(25, 401)
(10, 26)
(2,)


In [150]:
import scipy.optimize as op
result= op.minimize(fun= costFunc, x0= initTheta, method= 'TNC', jac= gradient)
result.x

ValueError: setting an array element with a sequence.